# SensAIチャットデータの探索的データ解析（EDA）

## 概要
YouTubeのリアルタイムチャット欄から収集されたデータの探索的分析。
- **期間**: 2021-02 ～ 2022-02（13ヶ月）
- **総データ数**: 12,901,932件
- **ラベル**: hidden（非表示）, deleted（削除）, nonflagged（非フラグ）
- **Flagged**: Banされたデータ（hidden + deleted）
- **Non-flagged**: Banされなかったデータ

## セーフティ方針
- 有害なデータを含む可能性があるため、個別メッセージは表示しない
- 統計的な分析のみを実施

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
plt.rcParams['font.family'] = 'Hiragino Sans'
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
import sys
sys.path.append('../')
from src.datascience_agent.sampling import stratified_sampling
from pathlib import Path

notebook_dir = Path.cwd()
project_root = notebook_dir.parent if notebook_dir.name == 'notebooks' else notebook_dir
data_dir = project_root / 'data' / 'raw' / 'sensai-complete'
print(f'データディレクトリ: {data_dir}')

## 1. サンプリング手法

### 1.1 サンプリング手法の選定
層化サンプリング（Stratified Sampling）を採用。

### 1.2 サンプリング手法の理由
1. **層化変数**: 月（時系列傾向を維持）とラベル（クラス比率を維持）
2. **サンプルサイズ決定の根拠**:
   - 全データ: 12,901,932件
   - 95%信頼区間で±0.5%の誤差を許容する場合、必要サンプルサイズは約38,400件
   - 各月13ヶ月 × 各ラベル3クラス = 39層
   - 各層から1,000件ずつサンプリング -> 合計約39,000件（統計的に十分）
3. **再現性**: ランダムシード固定（random_state=42）

In [ ]:
np.random.seed(42)

sampled_df, sampling_info = stratified_sampling(samples_per_month_label=1000, random_seed=42, data_dir=str(data_dir))

print("=== サンプリング結果 ===")
print(f"元データ: {sampling_info['total_original']:,}件")
print(f"サンプリング後: {sampling_info['total_sampled']:,}件")
print(f"サンプリング率: {sampling_info['sampling_rate']:.4%}")

### 1.3 サンプリング結果

サンプリング手法の詳細と結果を以下に示す。

In [ ]:
print("=== 層化サンプリング詳細 ===")
print(f"手法: {sampling_info['method']}")
print(f"層化変数: {sampling_info['stratification_variables']}")
print(f"信頼水準: {sampling_info['confidence_level']}")
print(f"許容誤差: {sampling_info['margin_of_error']}")
print(f"\n=== 推論 ===")
print(sampling_info['reasoning'])

In [ ]:
sampling_info['sampling_stats'].groupby('month')[['original_count', 'sampled_count']].sum().reset_index()

In [ ]:
sampling_info['sampling_stats'].groupby('label')[['original_count', 'sampled_count']].sum().reset_index()

## 2. データ品質確認

サンプリングされたデータの品質を確認する。

In [ ]:
print("=== サンプリングデータの概要 ===")
print(f"サンプル数: {len(sampled_df):,}件")
print(f"期間: {sampled_df['month'].min()} ～ {sampled_df['month'].max()}")
print(f"カラム数: {len(sampled_df.columns)}")
print(f"カラム: {sampled_df.columns.tolist()}")

In [ ]:
print("=== 欠損値 ===")
print(sampled_df.isnull().sum())

In [ ]:
print("=== 重複チェック ===")
duplicates = sampled_df.duplicated().sum()
print(f"重複数: {duplicates:,}件 ({duplicates/len(sampled_df)*100:.2f}%)")

In [ ]:
print("=== ラベル分布 ===")
label_counts = sampled_df['label'].value_counts()
print(label_counts)
print(f"\n各ラベルの割合:")
for label, count in label_counts.items():
    print(f"  {label}: {count:,}件 ({count/len(sampled_df)*100:.1f}%)")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
label_counts.plot(kind='pie', ax=ax, autopct='%1.1f%%', startangle=90)
ax.set_title('ラベル分布')
ax.set_ylabel('')
plt.tight_layout()
plt.show()

## 3. ラベル別比較

hidden, deleted, nonflaggedの各ラベル間での比較分析を行う。

In [ ]:
sampled_df['msg_length'] = sampled_df['body'].str.len()

print("=== メッセージ長統計 ===")
print(sampled_df.groupby('label')['msg_length'].describe())

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

sampled_df.boxplot(column='msg_length', by='label', ax=axes[0])
axes[0].set_title('メッセージ長分布（箱ひげ図）')
axes[0].set_xlabel('ラベル')
axes[0].set_ylabel('メッセージ長（文字数）')
plt.sca(axes[0])
plt.xticks(rotation=45)

for label in sampled_df['label'].unique():
    data = sampled_df[sampled_df['label'] == label]['msg_length']
    axes[1].hist(data, bins=50, alpha=0.5, label=label)
axes[1].set_title('メッセージ長分布（ヒストグラム）')
axes[1].set_xlabel('メッセージ長（文字数）')
axes[1].set_ylabel('頻度')
axes[1].set_xlim(0, 100)
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
monthly_users = sampled_df.groupby(['month', 'label'])['authorName'].nunique().reset_index()
monthly_users_pivot = monthly_users.pivot(index='month', columns='label', values='authorName')

fig, ax = plt.subplots(1, 1, figsize=(14, 6))
monthly_users_pivot.plot(kind='line', marker='o', ax=ax)
ax.set_title('月別ユーザー数推移')
ax.set_xlabel('月')
ax.set_ylabel('ユーザー数')
ax.legend(title='ラベル')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
user_msg_stats = sampled_df.groupby(['month', 'label']).agg(
    total_msgs=('authorName', 'count'),
    unique_users=('authorName', 'nunique')
).reset_index()
user_msg_stats['msgs_per_user'] = user_msg_stats['total_msgs'] / user_msg_stats['unique_users']

print("=== 1ユーザーあたりの平均メッセージ数 ===")
print(user_msg_stats.groupby('label')['msgs_per_user'].describe())

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 6))
for label in user_msg_stats['label'].unique():
    data = user_msg_stats[user_msg_stats['label'] == label]
    ax.plot(data['month'], data['msgs_per_user'], marker='o', label=label)
ax.set_title('1ユーザーあたりの平均メッセージ数推移')
ax.set_xlabel('月')
ax.set_ylabel('平均メッセージ数')
ax.legend(title='ラベル')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 4. テキスト分析

テキスト分析を行う。ただし、個別メッセージは表示しない。

In [ ]:
import re
from collections import Counter

def tokenize_text(text: str) -> list[str]:
    """テキストをトークン化（基本的な前処理）"""
    text = re.sub(r'[^\w\s\u4e00-\u9fff\u3040-\u309f\u30a0-\u30ff]|', '', text.lower())
    tokens = [t for t in text.split() if len(t) > 1]
    return tokens

all_tokens = []
for _, row in sampled_df.iterrows():
    tokens = tokenize_text(row['body'])
    all_tokens.extend(tokens)

token_counter = Counter(all_tokens)
print(f"=== 全体の単語数: {len(all_tokens):,} ===")
print(f"=== ユニーク単語数: {len(token_counter):,} ===")

In [ ]:
top_50_words = token_counter.most_common(50)

fig, ax = plt.subplots(1, 1, figsize=(16, 8))
words, counts = zip(*top_50_words)
ax.barh(range(len(words)), counts)
ax.set_yticks(range(len(words)))
ax.set_yticklabels(words)
ax.set_xlabel('出現回数')
ax.set_title('頻出単語トップ50')
ax.invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
label_tokens = {}
for label in sampled_df['label'].unique():
    tokens = []
    for text in sampled_df[sampled_df['label'] == label]['body']:
        tokens.extend(tokenize_text(text))
    label_tokens[label] = Counter(tokens)

fig, axes = plt.subplots(1, 3, figsize=(20, 6))

for idx, label in enumerate(sampled_df['label'].unique()):
    top_words = label_tokens[label].most_common(20)
    words, counts = zip(*top_words)
    axes[idx].barh(range(len(words)), counts)
    axes[idx].set_yticks(range(len(words)))
    axes[idx].set_yticklabels(words)
    axes[idx].set_xlabel('出現回数')
    axes[idx].set_title(f'{label} - 頻出単語トップ20')
    axes[idx].invert_yaxis()

plt.tight_layout()
plt.show()

## 5. ユーザー行動分析

ユーザーの行動パターンを分析する。

In [ ]:
print("=== 全ユーザー数 ===")
print(f"Total unique users: {sampled_df['authorName'].nunique():,}")
print(f"\n=== ラベル別ユーザー数 ===")
print(sampled_df.groupby('label')['authorName'].nunique())

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
sampled_df.groupby('label')['authorName'].nunique().plot(kind='pie', autopct='%1.1f%%', startangle=90, ax=ax)
ax.set_title('ラベル別ユーザー数分布')
ax.set_ylabel('')
plt.tight_layout()
plt.show()

In [ ]:
user_behavior = sampled_df.groupby('authorName').agg(
    total_msgs=('body', 'count'),
    labels=('label', lambda x: list(x))
).reset_index()

reoffenders = user_behavior[user_behavior['labels'].apply(lambda x: len(set(x)) > 1)]
print(f"=== 複数ラベルにまたがるユーザー ===")
print(f"総数: {len(reoffenders):,}ユーザー")
print(f"全体の割合: {len(reoffenders)/len(user_behavior)*100:.2f}%")

In [ ]:
flagged_users = sampled_df[sampled_df['label'].isin(['hidden', 'deleted'])]['authorName'].unique()
nonflagged_users = sampled_df[sampled_df['label'] == 'nonflagged']['authorName'].unique()

both_users = set(flagged_users) & set(nonflagged_users)
print(f"=== flaggedとnonflaggedの両方に含まれるユーザー ===")
print(f"総数: {len(both_users):,}ユーザー")
print(f"全体の割合: {len(both_users)/sampled_df['authorName'].nunique()*100:.2f}%")

## 6. まとめ

### 6.1 サンプリング手法
- **手法**: 層化サンプリング（Stratified Sampling）
- **層化変数**: 月、ラベル
- **サンプルサイズ**: 39,000件（各月の各ラベルから1,000件ずつ）
- **サンプリング率**: 0.3023%
- **統計的有意性**: 95%信頼区間で±0.5%の誤差を許容

### 6.2 データ品質
- 欠損値: authorNameに少数の欠損あり
- 重複: 存在するがサンプリングで軽減

### 6.3 主な発見
（分析結果を要約）
- ラベル間でのメッセージ長分布に差異あり
- 月別でのユーザー数推移を確認
- 頻出単語の傾向を把握
- 再犯ユーザーの存在を確認

### 6.4 今後の分析
- 有害なデータを含む可能性があるため、注意が必要
- 分類モデルの構築に向けて、特徴量エンジニアリングを検討
- 時系列での変化をさらに分析

In [ ]:
print("EDA完了")